<a href="https://colab.research.google.com/github/proninivandm/Agro_code/blob/main/Agrocode_%D0%9A%D1%83%D0%BB%D1%8C%D1%82%D1%83%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import catboost
RANDOM_SEED = 10

In [ ]:
def preproc(df):
  diffs_df = df[days_cols].diff(axis=1).drop('day_1', axis=1)
  residuals_df = days_mean_df - df[days_cols]
  residuals_med_df = days_median_df - df[days_cols]
  diffs_df.rename(columns=lambda x: x.replace('day', 'diff'), inplace=True)
  residuals_df.rename(columns=lambda x: x.replace('day', 'res'), inplace=True)
  residuals_med_df.rename(columns=lambda x: x.replace('day', 'res_med'), inplace=True)
  df = pd.concat([df, diffs_df, residuals_df, residuals_med_df], axis=1)
  df['vmax'] = df[days_cols].max(axis=1)
  df['vmin'] = df[days_cols].min(axis=1)
  df['dmax'] = diffs_df.max(axis=1)
  df['dmin'] = diffs_df.min(axis=1)
  df['rmax'] = residuals_df.max(axis=1)
  df['rmin'] = residuals_df.min(axis=1)
  df['rmmax'] = residuals_med_df.max(axis=1)
  df['rmmin'] = residuals_med_df.min(axis=1)
  df['vdiff'] = df.vmax-df.vmin
  df['ddiff'] = df.dmax-df.dmin
  df['rdiff'] = df.rmax-df.rmin
  df['rmdiff'] = df.rmmax-df.rmmin
  df['argvmax'] = df.apply(lambda x: x[days_cols].argmax(), axis=1)
  df['argvmin'] = df.apply(lambda x: x[days_cols].argmin(), axis=1)
  df['argdmax'] = diffs_df.apply(lambda x: x.argmax(), axis=1)
  df['argdmin'] = diffs_df.apply(lambda x: x.argmin(), axis=1)
  df['argrmax'] = residuals_df.apply(lambda x: x.argmax(), axis=1)
  df['argrmin'] = residuals_df.apply(lambda x: x.argmin(), axis=1)
  df['argrmmax'] = residuals_med_df.apply(lambda x: x.argmax(), axis=1)
  df['argrmmin'] = residuals_med_df.apply(lambda x: x.argmin(), axis=1)
  df['vmean'] = df.apply(lambda x: x[days_cols].mean(), axis=1)
  df['rmean'] = residuals_df.apply(lambda x: x.mean(), axis=1)
  df['rmmean'] = residuals_med_df.apply(lambda x: x.mean(), axis=1)
  df['vvar'] = df.apply(lambda x: x[days_cols].var(), axis=1)
  df['dvar'] = diffs_df.apply(lambda x: x.var(), axis=1)
  df['rvar'] = residuals_df.apply(lambda x: x.var(), axis=1)
  df['rmvar'] = residuals_med_df.apply(lambda x: x.var(), axis=1)
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')
example = pd.read_csv('/content/drive/MyDrive/example_csv_agrocode-task1.csv')

print(train.shape, test.shape)

(2838, 370) (939, 369)


In [ ]:
train.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
train = train.drop('day_366', axis=1)
test.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
test = test.drop('day_366', axis=1)

days_cols = [x for x in train.columns if 'day' in x]


In [ ]:
days_mean_df = pd.concat([train.drop('culture', axis=1), test], axis=0)[days_cols].apply(np.mean, axis=0)
days_median_df = pd.concat([train.drop('culture', axis=1), test], axis=0)[days_cols].apply(np.median, axis=0)

In [ ]:
to_fit=pd.get_dummies(pd.concat([train.drop('culture', axis=1), test], axis=0), columns=['year', 'field_id'])

In [ ]:
%%time
imputer = KNNImputer(n_neighbors=9, weights="distance")
imputer.fit(to_fit)

CPU times: user 48.4 ms, sys: 26.4 ms, total: 74.8 ms
Wall time: 79.3 ms


In [ ]:
to_fit[:len(train)] = imputer.transform(to_fit[:len(train)])
train[days_cols] = to_fit[:len(train)][days_cols]

In [ ]:
to_fit[-len(test):] = imputer.transform(to_fit[-len(test):])
test[days_cols] = to_fit[-len(test):][days_cols]

In [ ]:
train_new = preproc(train)
test_new = preproc(test)

In [ ]:
to_delete = [28,18,25,19,26,27]
train_new = train_new.drop(train_new.query('culture in {}'.format(to_delete)).index)

In [ ]:
un = train_new.culture.unique()

In [ ]:
x = range(len(un))

In [ ]:
labels = {sorted(un)[x]:x for x in x}

In [ ]:
labels

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 20: 17,
 21: 18,
 22: 19,
 23: 20,
 24: 21}

In [ ]:
train_new.culture=train_new.culture.apply(labels.get)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_new.drop('culture', axis=1), train_new.culture, random_state=RANDOM_SEED)

In [ ]:
train_pool = catboost.Pool(X_train, y_train, ['year', 'field_id'])
val_pool = catboost.Pool(X_val, y_val, ['year', 'field_id'])
test_pool = catboost.Pool(test_new, cat_features = ['year', 'field_id'])
full_pool = catboost.Pool(train_new.drop('culture', axis=1), train_new.culture, cat_features = ['year', 'field_id'])

In [ ]:
cat_model = catboost.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1:average=Weighted', classes_count=len(train_new.culture.unique()), task_type='GPU', random_state=RANDOM_SEED, iterations=2000)

In [ ]:
cat_model.fit(train_pool, eval_set=val_pool, use_best_model=True, metric_period=50)

In [ ]:
cat_model.fit(full_pool, metric_period=50)

Learning rate set to 0.045043
0:	learn: 0.5129877	total: 69.7ms	remaining: 2m 19s
50:	learn: 0.8097069	total: 3.6s	remaining: 2m 17s
100:	learn: 0.8675380	total: 6.93s	remaining: 2m 10s
150:	learn: 0.8974469	total: 10.1s	remaining: 2m 4s
200:	learn: 0.9199869	total: 13.2s	remaining: 1m 58s
250:	learn: 0.9366943	total: 16.3s	remaining: 1m 53s
300:	learn: 0.9506863	total: 19.4s	remaining: 1m 49s
350:	learn: 0.9577559	total: 22.4s	remaining: 1m 45s
400:	learn: 0.9680241	total: 25.5s	remaining: 1m 41s
450:	learn: 0.9716549	total: 28.5s	remaining: 1m 37s
500:	learn: 0.9786330	total: 31.5s	remaining: 1m 34s
550:	learn: 0.9838673	total: 34.6s	remaining: 1m 31s
600:	learn: 0.9873230	total: 37.6s	remaining: 1m 27s
650:	learn: 0.9892035	total: 40.6s	remaining: 1m 24s
700:	learn: 0.9906388	total: 43.6s	remaining: 1m 20s
750:	learn: 0.9913916	total: 46.7s	remaining: 1m 17s
800:	learn: 0.9932396	total: 49.7s	remaining: 1m 14s
850:	learn: 0.9951561	total: 52.7s	remaining: 1m 11s
900:	learn: 0.995156

In [ ]:
cv_model = catboost.cv(full_pool, params = cat_model.get_params(), fold_count=5, partition_random_seed=RANDOM_SEED, metric_period = 50, plot=True)

In [ ]:
cv_model

,iterations,test-TotalF1:average=Weighted-mean,test-TotalF1:average=Weighted-std,train-TotalF1:average=Weighted-mean,train-TotalF1:average=Weighted-std,test-MultiClass-mean,test-MultiClass-std,train-MultiClass-mean,train-MultiClass-std
0,0,0.447672,0.057025,0.510973,0.003724,2.994055,0.026986,2.936852,0.007742
1,50,0.716477,0.014413,0.765940,0.005688,1.131360,0.032245,1.024015,0.006218
2,100,0.757519,0.019840,0.832132,0.007606,0.844550,0.026860,0.705798,0.006149
3,150,0.785634,0.025300,0.867254,0.006985,0.738326,0.026934,0.572996,0.005352
4,200,0.802759,0.019291,0.889445,0.006436,0.676416,0.026008,0.489646,0.004656
5,250,0.813601,0.015779,0.904525,0.003882,0.633052,0.025754,0.428618,0.003955
6,300,0.823981,0.016130,0.919117,0.002660,0.601287,0.026296,0.382132,0.002879
7,350,0.831690,0.016551,0.931622,0.003623,0.576720,0.024922,0.344163,0.002852
8,400,0.837467,0.014399,0.941510,0.002687,0.556282,0.024525,0.312705,0.002896
9,450,0.842106,0.012348,0.951222,0.002140,0.539810,0.024350,0.286775,0.003276


In [ ]:
train_new.culture.value_counts()

In [ ]:
train.groupby('culture').mean()

In [ ]:
0.876496

In [ ]:
test['Culture'] = cat_model.predict(test_pool)

In [ ]:
labels = {k:v for v,k in labels.items()}

In [ ]:
labels

In [ ]:
test.Culture = test.Culture.apply(labels.get)

In [ ]:
test.to_csv('preds9_cfull_pool.csv', index=False, columns=['Culture'])

In [ ]:
len(test)